In [3]:
import json
import time
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import cv2
from pathlib import Path


# Getting File

In [5]:
# Opening JSON file
start = time.time()
with open('dict_spottings.json') as json_file:
    data = dict(json.load(json_file))

In [7]:
def removeType(string):
    return string[:-4]

In [8]:
videoLocation = 'D:/BOBSL/BOBSL200G/bobsl/videos'
videoTitles = os.listdir(videoLocation)

videoName = videoTitles[0]
targetVideo = videoLocation + '/' +videoName

videoTitlesEdited = list(map(removeType, videoTitles))

# Only Loading in files that I have Downloaded

In [9]:
wordsTrain = list(dict(data['train']).keys())

train = []
for word in wordsTrain:
    wordData = data['train'][word]
    for i in range(len(wordData['names'])):
        fileName = wordData['names'][i]
        if fileName in videoTitlesEdited:
            globalTime = wordData['global_times'][i]
            probs = wordData['probs'][i]
            train.append((word, fileName, globalTime, probs))

KeyboardInterrupt: 

In [ ]:
wordsVal = list(dict(data['val']).keys())

val = []
for word in wordsVal:
    wordData = data['val'][word]
    for i in range(len(wordData['names'])):
        fileName = wordData['names'][i]
        if fileName in videoTitlesEdited:
            globalTime = wordData['global_times'][i]
            probs = wordData['probs'][i]
            val.append((word, fileName, globalTime, probs))

In [4]:
wordsTest = list(dict(data['test']).keys())

test = []
for word in wordsTest:
    wordData = data['test'][word]
    for i in range(len(wordData['names'])):
        fileName = wordData['names'][i]
        if fileName in videoTitlesEdited:
            globalTime = wordData['global_times'][i]
            probs = wordData['probs'][i]
            test.append((word, fileName, globalTime, probs))

NameError: name 'videoTitlesEdited' is not defined

In [ ]:
print(len(train))
print(len(val))
print(len(test))

## Create new file showing information about files i have

In [ ]:
f1 = open("train.txt", "w")
for line in train:
    f1.write(str(line) + '\n')
f1.close()

f2 = open("val.txt", "w")
for line in val:
    f2.write(str(line) + '\n')
f2.close()

f3 = open("test.txt", "w")
for line in test:
    f3.write(str(line) + '\n')
f3.close()

# Creating smaller videos

In [ ]:
sortedVideoList = []
for videoName in videoTitlesEdited:
    targeVideoInfo = []
    for wordInfo in val: # wordInfo = (word, fileName, globalTime, probs)
        if wordInfo[1] == videoName:
            targeVideoInfo.append(wordInfo)
    if len(targeVideoInfo) > 0:  
        sortedVideoList.append(targeVideoInfo)
sortedVideoList

In [ ]:
video1 = sortedVideoList[0]
FPS = 25
parts = []
for info in video1:
    globalTime = info[2]
    frameStart = globalTime
    frameEnd = globalTime + 1.5
    parts.append((frameStart, frameEnd, info[0]))
parts = sorted(parts, key=lambda x: x[0])
f4 = open("check2.txt", "w")
for line in parts:
    f4.write(str(line) + '\n')
f4.close()

In [ ]:

videoLocation = 'D:/BOBSL/BOBSL200G/bobsl/videos'
videoTitles = os.listdir(videoLocation)

saveLocation = 'D:/Thesis/Video-Segments'

videoName = video1[0][1]

In [84]:
def extract_segment(input_file, start_time, end_time, output_file):
    cap = cv2.VideoCapture(input_file)

    # Get video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Calculate start and end frame
    start_frame = int(start_time * fps)
    end_frame = int(end_time * fps)

    # Set video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_file, fourcc, fps, (width, height))

    # Extract frames and write to output file
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame_num = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
        if frame_num < start_frame:
            continue
        if frame_num > end_frame:
            break

        out.write(frame)

    # Release resources
    cap.release()
    out.release()

In [ ]:
videoName = video1[0][1]
targetVideo = videoLocation + '/' + videoName + '.mp4'
saveLocation = 'D:/Thesis/Video-Segments'
j = 0
for start, end, word in parts:
    if j == 250:
        break
    
    output  = saveLocation + '/' + videoName + '-' + word + '-'+ str(int(start)) +'-' + str(int(end)) + '.mp4'
    path = Path(output)
    i = 1
    while(path.is_file()):
        continue
    print(output, start, end, word)
    extract_segment(targetVideo, start, end, output)
    j += 1


# ELAR Dataset

In [10]:
from pympi import Eaf

### Function to extract signs from a video

In [11]:
def extract_segment_adpated(input_file, timesWord, output_loc, videoName):
    cap = cv2.VideoCapture(input_file)
    
    # Get video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Extract frames and write to output file
    frames = {}
    print('video: start frame')
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame_num = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
        frames[frame_num] = frame

    print('video: end frames')
    # Release resources
    cap.release()


    i = 0
    print('Start: creating segments')
    for start, end, word in timesWord:
        # print(i, start, end, word)
        output_file = output_loc + word +'_'+ videoName +'_' + str(int(start)) + '_' + str(int(end)) + '.mp4'

        # Calculate start and end frame
        start_frame = int(start * fps * 0.001)
        end_frame = int(end * fps * 0.001)

        # Set video writer
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_file, fourcc, fps, (width, height))
        for frame_num in range(start_frame, end_frame):
            try:
                frame = frames[frame_num]
            except KeyError as error:
                continue
            out.write(frame)
        out.release()
        i += 1
    print('Finished: creating segments')

### Create segments by sign for one video file

In [ ]:
videoName = 'AAM1A1c2a'
videoLocation = 'D:/Auslan/AUSLANRECO-I0569/data/ELAR Data/AAM1A1c2a - The Boy Who Cried Wolf'
targetVideo = videoLocation + '/' + videoName + '.mp4'
saveLocation = 'D:/Thesis/Video-Segments-AUSLAN'

fileEAF = 'D:/Auslan/AUSLANRECO-I0569/data/ELAR Data/AAM1A1c2a - The Boy Who Cried Wolf/AAM1A1c2a.eaf'
eaf = Eaf(fileEAF)

j = 0
timeWords = []
for key in list(eaf.tiers['RH-IDgloss'][0].keys()):
    if j == 250:
        break
    
    start = eaf.timeslots[eaf.tiers['RH-IDgloss'][0][key][0]]
    end = eaf.timeslots[eaf.tiers['RH-IDgloss'][0][key][1]]
    word = eaf.tiers['RH-IDgloss'][0][key][2]
    timeWords.append((start, end, word))
print('Words times finished')

extract_segment_adpated(targetVideo, timeWords, saveLocation + '/', videoName)


### Convert All video Files

In [12]:
folderLocation = 'D:/Auslan/AUSLANRECO-I0569/data/ELAR Data/'
folderList = os.listdir(folderLocation)
saveLocationParent = 'D:/Thesis/Video-Segments-AUSLAN/' 

i = 0
for folderName in folderList:
    if folderName == '! Annotated Files':
        continue
    # Video File 
    videoLocation = folderLocation + folderName
    videoName = folderName.split()[0]
    targetVideo = videoLocation + '/' + videoName + '.mp4'
    # ELAR File
    fileEAF = videoLocation + '/' + videoName + '.eaf'
    try:
        eaf = Eaf(fileEAF)
    except FileNotFoundError as error:
        continue
    # Save Location
    path = os.path.join(saveLocationParent, videoName)
    try:
        os.mkdir(path)
    except OSError as error:
        print(error)

    saveLocation = saveLocationParent + videoName
    print(saveLocation)
    timeWords = []
    for key in list(eaf.tiers['RH-IDgloss'][0].keys()):
        start = eaf.timeslots[eaf.tiers['RH-IDgloss'][0][key][0]]
        end = eaf.timeslots[eaf.tiers['RH-IDgloss'][0][key][1]]
        word = eaf.tiers['RH-IDgloss'][0][key][2]
        timeWords.append((start, end, word))
    print('Words times finished')
    extract_segment_adpated(targetVideo, timeWords, saveLocation + '/', videoName)
    i+=1

D:/Thesis/Video-Segments-AUSLAN/AAM1A1c2a
Words times finished
video: start frame
video: end frames
Start: creating segments
Finished: creating segments
D:/Thesis/Video-Segments-AUSLAN/AAM1A2c6ii
Words times finished
video: start frame
video: end frames
Start: creating segments
Finished: creating segments
D:/Thesis/Video-Segments-AUSLAN/AAM1A2c6iv
Words times finished
video: start frame
video: end frames
Start: creating segments
Finished: creating segments
D:/Thesis/Video-Segments-AUSLAN/AAM2A1c2b
Words times finished
video: start frame
video: end frames
Start: creating segments
Finished: creating segments
D:/Thesis/Video-Segments-AUSLAN/AAM2A2c6ii
Words times finished
video: start frame
video: end frames
Start: creating segments
Finished: creating segments
D:/Thesis/Video-Segments-AUSLAN/AAM2A2c6iv
Words times finished
video: start frame
video: end frames
Start: creating segments
Finished: creating segments
D:/Thesis/Video-Segments-AUSLAN/AAPB1c2b
Words times finished
video: start fra

C:\Users\KAI\AppData\Roaming\Python\Python39\site-packages\pympi\Elan.py:1471: UserWarning: Parsing unknown version of ELAN spec... This could result in errors...
  warnings.warn('Parsing unknown version of ELAN spec... '


video: end frames
Start: creating segments
Finished: creating segments
D:/Thesis/Video-Segments-AUSLAN/AMW1B2c6iii
Words times finished
video: start frame
video: end frames
Start: creating segments
Finished: creating segments
D:/Thesis/Video-Segments-AUSLAN/AMW1B2c7a
Words times finished
video: start frame
video: end frames
Start: creating segments
Finished: creating segments
D:/Thesis/Video-Segments-AUSLAN/AMW2A1c2b
Words times finished
video: start frame
video: end frames
Start: creating segments
Finished: creating segments
D:/Thesis/Video-Segments-AUSLAN/AMW2A2c6ii
Words times finished
video: start frame
video: end frames
Start: creating segments
Finished: creating segments
D:/Thesis/Video-Segments-AUSLAN/AMW2A2c7a
Words times finished
video: start frame
video: end frames
Start: creating segments
Finished: creating segments
D:/Thesis/Video-Segments-AUSLAN/ARGB2c6iii
Words times finished
video: start frame
video: end frames
Start: creating segments
Finished: creating segments
D:/Thes

### Test Stuff

In [85]:
output  = saveLocation + '/' + 'test' + '-' + word + '-'+ str(int(start)) +'-' + str(int(end)) + '.mp4'
extract_segment(targetVideo, 100, 110, output)

### How many include both Subtitles

In [19]:
folderLocation = 'D:/Auslan/AUSLANRECO-I0569/data/ELAR Data/'
folderList = os.listdir(folderLocation)
i=0
j=0
for folderName in folderList:
        j+=1
        if folderName == '! Annotated Files':
                continue
        videoName = folderName.split()[0]
        videoLocation = folderLocation + folderName
        # print(folderLocation + folderName)
        # ELAR File
        fileEAF = videoLocation + '/' + videoName + '.eaf'
        # print(fileEAF)
        try:
                eaf = Eaf(fileEAF)
        except FileNotFoundError as error:
                continue
        if len(eaf.tiers['FreeTransl'][0]):
                i += 1
print(i)
print(j)


C:\Users\KAI\AppData\Roaming\Python\Python39\site-packages\pympi\Elan.py:1471: UserWarning: Parsing unknown version of ELAN spec... This could result in errors...
  warnings.warn('Parsing unknown version of ELAN spec... '


103
211
